In [9]:
import itertools
import torch
import torch.optim as optim
import torchvision
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Function
from torchsummary import summary
import os
import numpy as np
from collections import defaultdict
import copy

In [10]:
class ReverseLayer(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [11]:
def build_encoder():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34')
    modules = list(model.children())[:-1] + [torch.nn.Flatten()]
    model = nn.Sequential(*modules)
    return model

def build_classifier(input_shape, classes, joint=True):
    domains = 2 if joint else 1
    
    classifier = nn.Sequential(
        nn.Linear(input_shape, 1280),
        nn.ReLU(True),
        nn.Dropout(0.2),
        nn.Linear(1280, 1280),
        nn.ReLU(True),
        nn.Dropout(0.2),
        nn.Linear(1280, domains*classes),
        nn.Softmax()
    )
    return classifier

def build_discriminator(input_shape):
    discriminator = nn.Sequential(
        nn.Linear(input_shape, 1280),
        nn.ReLU(True),
        nn.Linear(1280, 1280),
        nn.ReLU(True),
        nn.Linear(1280, 2),
        nn.Softmax()
    )
    return discriminator

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels=True):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index):
        x = self.data[index]
        return x if self.labels else x[0]
    
    def __len__(self):
        return len(self.data)

In [28]:
class DataGenerator:
    def __init__(
        self, 
        source_domain,
        target_domain,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.1,
        target_train=False
    ):
        src_train, src_val = self.__prepare_data(
            source_domain, 
            input_shape,
            True, 
            val_split,
            test_split,
            target_labels
        )
              
        train_label, train_nlabel, tar_val, test = self.__prepare_data(
            target_domain,  
            input_shape, 
            False, 
            val_split,
            test_split,
            target_labels
        )
        
        if len(tar_val) <= 10:
            val = torch.utils.data.ConcatDataset([src_val, tar_val])
        else:
            val = tar_val
        
#         if target_train:
#             train_label_set = train_label
#         else:
#             train_label_set = torch.utils.data.ConcatDataset([src_train, train_label])

        self.train_src_loader = DataLoader(dataset=src_train, batch_size=32, shuffle=True, num_workers=0)
        self.train_tar_label_loader = DataLoader(dataset=train_label, batch_size=32, shuffle=True, num_workers=0)
        
        if target_labels < 1:
            self.train_tar_nlabel_loader = DataLoader(dataset=train_nlabel, batch_size=32, shuffle=True, num_workers=0)
        else:
            self.train_tar_nlabel_loader = None
            
        self.val_loader = DataLoader(dataset=val, batch_size=64, shuffle=True, num_workers=0)
        self.test_loader = DataLoader(dataset=test, batch_size=64, shuffle=True, num_workers=0)
    
    def __prepare_data(self, folder, input_shape, src=True, val_split=0, test_split=0, target_labels=0.1):
        
        transform = transforms.Compose([
            transforms.Resize(input_shape),
            transforms.ToTensor()
        ])
        
        label = 0 if src else 1
        
        data = torchvision.datasets.ImageFolder(folder, transform=transform)
        data.target_transform = lambda id: torch.Tensor((label, id))
        
        self.classes = data.classes
        
        if src:
            
            train, val = torch.utils.data.random_split(
                data, 
                [round(len(data)*(1-val_split) - 1e-5), round(len(data)*val_split + 1e-5)]
            ) 
            
            return train, val
        
        else:
            data, test = torch.utils.data.random_split(
                data, 
                [round(len(data)*(1-test_split) - 1e-5), round(len(data)*test_split + 1e-5)]
            )
        
            train, train_nlabel = torch.utils.data.random_split(
                data, 
                [round(len(data)*target_labels - 1e-5), round(len(data)*(1-target_labels) + 1e-5)]
            ) 
    
            train_label, val = torch.utils.data.random_split(
                train, 
                [round(len(train)*(1-val_split) - 1e-5), round(len(train)*val_split + 1e-5)]
            )
            
            return train_label, Dataset(train_nlabel, False), val, test
        
    def train_data(self):
        return train_src_loader, self.train_tar_label_loader, self.train_tar_nlabel_loader
    
    def val_data(self):
        return self.val_loader
    
    def test_data(self):
        return self.test_loader

In [ ]:
data_folder = "../Datasets/Experiment"
src_folder = os.path.join(data_folder, "Real World")
target_folder = os.path.join(data_folder, "Product")

d = DataGenerator(
    source_domain=src_folder,
    target_domain=target_folder,
    val_split=0.2,
    test_split=0.2,
    input_shape=(224,224),
    target_labels=0.01,
    target_train=False
)

train_label, train_nlabel = d.train_data()
val = d.val_data()
test = d.test_data()

In [14]:
def KL_Loss(y_pred, classes):
    y_joint = torch.reshape(y_pred, (-1, 2*classes))
    
    y_class = torch.unsqueeze(torch.sum(y_pred, 1), 1)
    y_domain = torch.unsqueeze(torch.sum(y_pred, 2), -1)
    
    
    y_ind_joint = torch.reshape((y_domain * y_class), (-1,2*classes))
    
    return torch.nn.KLDivLoss(log_target=True, reduction="sum")(
        torch.log(y_joint), 
        torch.log(y_ind_joint)
    )

In [7]:
class DANNModel(nn.Module):
    def __init__(
        self, 
        classes=65,
    ):
        super(SingleDomainModel, self).__init__()
        self.classes = classes
        self.encoder = build_encoder()
        self.classifier = build_classifier(512, self.classes)
        self.discriminator = build_discriminator(512)
    
    def forward(self, inputs, rep_loss):
        features = self.encoder(inputs)
        classes = self.classifier(features)
        classes = torch.reshape(classes, (-1, 2, self.classes))
        domains = self.discriminator(ReverseLayer.apply(features, rep_loss))
        return classes, domains
    
    def run_batch(self, imgs, labels=None, training=True, use_KL=True, rep_loss=1, risk_loss=1):
        if training:
            opt = optim.Adam(self.parameters(), lr=1e-4)
#             opt = torch.optim.SGD(m.parameters(), lr=1e-4, momentum=0.9, weight_decay=5e-4)
            opt.zero_grad()
            
        kl_loss = 0
        class_pred_loss = 0
        pred_count_joint = 0
        pred_count_class = 0
        
        imgs = imgs.cuda()
        y_pred, domain_pred_label = self(imgs, rep_loss) 
        
        if labels is not None:
            labels = labels.cuda()
            labels = labels.to(dtype=torch.int64)
            joint_labels = labels[:,0] * m.classes + labels[:,1]
            dom_labels = torch.cat((labels[:,0], torch.ones(imgs.shape[0] - labels.shape[0], dtype=torch.long).cuda()), 0)

            y_pred = y_pred[:labels.shape[0]]
            y_joint = torch.reshape(y_pred, (-1,2*m.classes))
            y_class = torch.sum(y_pred, 1)

            if use_KL:
                kl_loss = KL_Loss(y_pred, m.classes) * risk_loss

            class_pred_loss = torch.nn.NLLLoss()(torch.log(y_class), labels[:,1])
            
            pred_count_joint = torch.count_nonzero(torch.argmax(y_joint, 1) == joint_labels)
            pred_count_class = torch.count_nonzero(torch.argmax(y_class, 1) == labels[:,1])
             
        dis_loss = torch.nn.NLLLoss()(torch.log(domain_pred_label), dom_labels)
        dis_count = torch.count_nonzero(torch.argmax(domain_pred_label, 1) == dom_labels)
        
        loss = class_pred_loss + kl_loss + dis_loss

        if training:
            loss.backward()
            opt.step()
        
        return (
            float(loss), 
            float(kl_loss), 
            float(class_pred_loss), 
            float(dis_loss), 
            float(pred_count_joint), 
            float(pred_count_class),
            float(dis_count)
        )

In [ ]:
class DANNModel(nn.Module):
    def __init__(
        self, 
        classes=65,
        use_KL=True,
        risk_lambda=1,
        rep_lambda=1
        lr=1e-4
    ):
        super(DANNModel, self).__init__()
        self.classes = classes
        self.encoder = build_encoder()
        self.classifier = build_classifier(512, self.classes, use_KL)
        self.discriminator = build_discriminator(512)
        
        self.lr = lr
        
        self.use_KL = use_KL
        self.risk_lambda = risk_lambda
        self.rep_lambda = rep_lambda
        
    def forward(self, inputs):
        features = self.encoder(inputs)
        classes = self.classifier(features)
        if self.use_KL:
            classes = torch.reshape(classes, (-1, 2, self.classes))
        domains = self.discriminator(ReverseLayer.apply(features, rep_loss))
        return classes, domains
    
    def __print_metrics(self, metrics, prefix="", precision=4, space=1):
        for k, v in metrics.items():
            print(prefix + k, round(v, precision), sep=": ", end=' ')
        for i in range(space):
            print()
    
    def __train_encoder_discriminator(self, imgs, y_labels, domain_labels):
        opt = optim.Adam(self.encoder.parameters() + self.discriminator.parameters(), lr=self.lr)
        opt.zero_grad()
        
        # Feedforward
        y_pred, domain_pred = self(imgs) 
        y_class = torch.sum(y_pred, 1)
        
        # Calculate losses and metrics
        class_pred_loss = torch.nn.NLLLoss()(torch.log(y_class), y_labels)
        kl_loss = KL_Loss(y_pred, m.classes) * self.risk_lambda
        dis_loss = torch.nn.NLLLoss()(torch.log(domain_pred), domain_labels) * self.rep_lambda
        
        loss = class_pred_loss + kl_loss + dis_loss
        
        # Backpropogation
        loss.backward()
        opt.step()
        
        return y_class, float(loss), float(kl_loss), float(dis_loss)
            
    def __train_classifier(self, imgs, labels):
        classifier_opt = optim.Adam(self.encoder.parameters(), lr=self.lr)
        classifier_opt.zero_grad()
        
        # Feedforward
        y_pred, _ = self(imgs) 
        y_joint = torch.reshape(y_pred, (-1,2*m.classes))          

        # Calculate losses and metrics
        joint_pred_loss = torch.nn.NLLLoss()(torch.log(y_joint), labels)
        classifier_loss = joint_pred_loss
        
        # Backpropogation
        classifier_loss.backward()
        classifier_opt.step()
        
        return y_pred, float(classifier_loss)
    
    def __full_training(self, imgs, y_labels, domain_labels):
        opt = optim.Adam(self.parameters(), lr=self.lr)
        opt.zero_grad()
        
        # Feedforward
        y_pred, domain_pred = self(imgs) 
        
        # Calculate losses and metrics
        class_pred_loss = torch.nn.NLLLoss()(torch.log(y_pred), labels)
        dis_loss = torch.nn.NLLLoss()(torch.log(domain_pred), domain_labels) * self.rep_lambda
        loss = class_pred_loss + dis_loss
        
        # Backpropogation
        loss.backward()
        opt.step()
        
        return y_pred, float(loss), float(dis_loss)
    
    def __train_batch(self, imgs, labels=None):
        
        imgs = imgs.cuda()
        
        if labels is not None:
            labels = labels.cuda()
            labels = labels.to(dtype=torch.int64)
            class_labels = labels[:,1]
            joint_labels = labels[:,0] * m.classes + labels[:,1]
            domain_labels = torch.cat((labels[:,0], torch.ones(imgs.shape[0] - labels.shape[0], dtype=torch.long).cuda()), 0)
        
        if self.use_KL:
            # Two step training of encoder/discriminator and classifier with KL-loss
            _, encoder_loss, kl_loss, dis_loss = self.__train_encoder_discriminator(imgs, class_labels, domain_labels)
            y_pred, classifier_loss = self.__train_classifier(imgs, joint_labels)
            
            y_class = torch.sum(y_pred, 1)
            y_joint = torch.reshape(y_pred, (-1,2*m.classes))
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_class, 1) == class_labels)
            pred_joint_count = torch.count_nonzero(torch.argmax(y_joint, 1) == joint_labels)
            pred_dis_count = torch.count_nonzero(torch.argmax(y_joint, 1) == joint_labels)
            
            loss_metrics = {
                "loss": encoder_loss + classifier_loss,
                "encoder_loss": encoder_loss,
                "kl_loss": kl_loss,
                "disciminator_loss": dis_loss,
                "classifier_loss": classifier_loss
            }
            
            acc_metrics = {
                "joint_acc": float(pred_joint_count),
                "class_acc": float(pred_class_count)
            }
            
        else:
            y_pred, loss = self.__full_training(imgs, class_labels)
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_pred, 1) == class_labels)
            
            loss_metrics = {
                "loss": loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }

        return loss_metrics, acc_metrics
    
    def train(self, epochs, train_loader, val_loader=None, patience=None, verbose=True):
        
        train_size = len(train_loader.dataset)
        min_val_loss = np.inf
        best_model = None
        
        current_patience = patience
        
        # Perform training over number of epochs
        for i in range(epochs):
            if verbose:
                print(f"Epoch {i+1}")
            
            loss_metrics = defaultdict(float)
            acc_metrics = defaultdict(float)
            
            for imgs, labels in train_loader:
                batch_loss, batch_acc = self.__train_batch(imgs, labels)

                for k, v in batch_loss.items():
                    loss_metrics[k] += v / train_size
                for k, v in batch_acc.items():
                    acc_metrics[k] += v / train_size
                    
            if verbose:
                self.__print_metrics(loss_metrics, precision=6)
                self.__print_metrics(acc_metrics, precision=4, space=2)
            
            # Run on validation set if provided
            if val_loader is not None:
                val_size = len(val_loader.dataset)
                loss_metrics, acc_metrics = self.evaluate(val_loader, val=True, verbose=verbose)
                
                # Early stopping
                if loss_metrics["loss"] < min_val_loss:
                    min_val_loss = loss_metrics["loss"]
                    best_model = copy.deepcopy(self.state_dict())
                    current_patience = patience
                else:
                    if current_patience is not None:
                        current_patience -= 1
                        if current_patience <= 0:
                            break
                        
        self.load_state_dict(best_model)
    
    def evaluate(self, loader, val=False, verbose=True):
        
        data_size = len(loader.dataset)
        loss_metrics = defaultdict(float)
        acc_metrics = defaultdict(float)
        
        for imgs, labels in loader:
            batch_loss, batch_acc = self.__evaluate_batch(imgs, labels)
            
            for k, v in batch_loss.items():
                loss_metrics[k] += v / data_size
            for k, v in batch_acc.items():
                acc_metrics[k] += v / data_size
                
        prefix = "val_" if val else ""
        
        if verbose:
            self.__print_metrics(loss_metrics, prefix=prefix, precision=6)
            self.__print_metrics(acc_metrics, prefix=prefix, precision=4, space=2)
                
        return loss_metrics, acc_metrics
            
    def __evaluate_batch(self, imgs, labels):
        imgs = imgs.cuda()
        labels = labels.cuda()
        labels = labels.to(dtype=torch.int64)
        class_labels = labels[:,1]
        joint_labels = labels[:,0] * m.classes + labels[:,1]
        
        # Feedforward
        y_pred = self(imgs)
        
        if self.use_KL:           
            y_class = torch.sum(y_pred, 1)
            y_joint = torch.reshape(y_pred, (-1,2*m.classes))
            
            class_pred_loss = float(torch.nn.NLLLoss()(torch.log(y_class), class_labels))
            joint_pred_loss = float(torch.nn.NLLLoss()(torch.log(y_joint), joint_labels))
            kl_loss = float(KL_Loss(y_pred, m.classes) * self.risk_lambda)
            
            encoder_loss = class_pred_loss + kl_loss
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_class, 1) == class_labels)
            
            loss_metrics = {
                "loss": encoder_loss,
                "kl_loss": kl_loss,
                "classifier_loss": class_pred_loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }
            
        else:          
            pred_class_count = torch.count_nonzero(torch.argmax(y_pred, 1) == class_labels)
            loss = float(torch.nn.NLLLoss()(torch.log(y_pred), class_labels))
            
            loss_metrics = {
                "loss": loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }

        return loss_metrics, acc_metrics

In [15]:
class SingleDomainModel(nn.Module):
    def __init__(
        self, 
        classes=65,
        use_KL=True,
        risk_lambda=1,
        lr=1e-4
    ):
        super(SingleDomainModel, self).__init__()
        self.classes = classes
        self.encoder = build_encoder()
        self.classifier = build_classifier(512, self.classes, use_KL)
        
        self.lr = lr
        
        self.use_KL = use_KL
        self.risk_lambda = risk_lambda
        
    def forward(self, inputs):
        features = self.encoder(inputs)
        classes = self.classifier(features)
        if self.use_KL:
            classes = torch.reshape(classes, (-1, 2, self.classes))
        return classes
    
    def __print_metrics(self, metrics, prefix="", precision=4, space=1):
        for k, v in metrics.items():
            print(prefix + k, round(v, precision), sep=": ", end=' ')
        for i in range(space):
            print()
    
    def __train_encoder(self, imgs, labels):
        encoder_opt = optim.Adam(self.encoder.parameters(), lr=self.lr)
        encoder_opt.zero_grad()
        
        # Feedforward
        y_pred = self(imgs) 
        y_class = torch.sum(y_pred, 1)
        
        # Calculate losses and metrics
        class_pred_loss = torch.nn.NLLLoss()(torch.log(y_class), labels)
        kl_loss = KL_Loss(y_pred, m.classes) * self.risk_lambda
        encoder_loss = class_pred_loss + kl_loss
        
        # Backpropogation
        encoder_loss.backward()
        encoder_opt.step()
        
        return y_class, float(encoder_loss), float(kl_loss)
            
    def __train_classifier(self, imgs, labels):
        classifier_opt = optim.Adam(self.encoder.parameters(), lr=self.lr)
        classifier_opt.zero_grad()
        
        # Feedforward
        y_pred = self(imgs) 
        y_joint = torch.reshape(y_pred, (-1,2*m.classes))          

        # Calculate losses and metrics
        joint_pred_loss = torch.nn.NLLLoss()(torch.log(y_joint), labels)
        classifier_loss = joint_pred_loss
        
        # Backpropogation
        classifier_loss.backward()
        classifier_opt.step()
        
        return y_pred, float(classifier_loss)
    
    def __full_training(self, imgs, labels):
        opt = optim.Adam(self.parameters(), lr=self.lr)
        opt.zero_grad()
        
        # Feedforward
        y_pred = self(imgs) 
        
        # Calculate losses and metrics
        class_pred_loss = torch.nn.NLLLoss()(torch.log(y_pred), labels)
        loss = class_pred_loss
        
        # Backpropogation
        loss.backward()
        opt.step()
        
        return y_pred, float(loss)
    
    def __train_batch(self, imgs, labels):
        
        imgs = imgs.cuda()
        labels = labels.cuda()
        labels = labels.to(dtype=torch.int64)
        class_labels = labels[:,1]
        joint_labels = labels[:,0] * m.classes + labels[:,1]
        
        if self.use_KL:
            # Two step training of encoder and classifier with KL-loss
            _, encoder_loss, kl_loss = self.__train_encoder(imgs, class_labels)
            y_pred, classifier_loss = self.__train_classifier(imgs, joint_labels)
            
            y_class = torch.sum(y_pred, 1)
            y_joint = torch.reshape(y_pred, (-1,2*m.classes))
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_class, 1) == class_labels)
            pred_joint_count = torch.count_nonzero(torch.argmax(y_joint, 1) == joint_labels)
            
            loss_metrics = {
                "loss": encoder_loss + classifier_loss,
                "encoder_loss": encoder_loss,
                "kl_loss": kl_loss,
                "classifier_loss": classifier_loss
            }
            
            acc_metrics = {
                "joint_acc": float(pred_joint_count),
                "class_acc": float(pred_class_count)
            }
            
        else:
            y_pred, loss = self.__full_training(imgs, class_labels)
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_pred, 1) == class_labels)
            
            loss_metrics = {
                "loss": loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }

        return loss_metrics, acc_metrics
    
    def train(self, epochs, train_loader, val_loader=None, patience=None, verbose=True):
        
        train_size = len(train_loader.dataset)
        min_val_loss = np.inf
        best_model = None
        
        current_patience = patience
        
        # Perform training over number of epochs
        for i in range(epochs):
            if verbose:
                print(f"Epoch {i+1}")
            
            loss_metrics = defaultdict(float)
            acc_metrics = defaultdict(float)
            
            for imgs, labels in train_loader:
                batch_loss, batch_acc = self.__train_batch(imgs, labels)

                for k, v in batch_loss.items():
                    loss_metrics[k] += v / train_size
                for k, v in batch_acc.items():
                    acc_metrics[k] += v / train_size
                    
            if verbose:
                self.__print_metrics(loss_metrics, precision=6)
                self.__print_metrics(acc_metrics, precision=4, space=2)
            
            # Run on validation set if provided
            if val_loader is not None:
                val_size = len(val_loader.dataset)
                loss_metrics, acc_metrics = self.evaluate(val_loader, val=True, verbose=verbose)
                
                # Early stopping
                if loss_metrics["loss"] < min_val_loss:
                    min_val_loss = loss_metrics["loss"]
                    best_model = copy.deepcopy(self.state_dict())
                    current_patience = patience
                else:
                    if current_patience is not None:
                        current_patience -= 1
                        if current_patience <= 0:
                            break
                        
        self.load_state_dict(best_model)
    
    def evaluate(self, loader, val=False, verbose=True):
        
        data_size = len(loader.dataset)
        loss_metrics = defaultdict(float)
        acc_metrics = defaultdict(float)
        
        for imgs, labels in loader:
            batch_loss, batch_acc = self.__evaluate_batch(imgs, labels)
            
            for k, v in batch_loss.items():
                loss_metrics[k] += v / data_size
            for k, v in batch_acc.items():
                acc_metrics[k] += v / data_size
                
        prefix = "val_" if val else ""
        
        if verbose:
            self.__print_metrics(loss_metrics, prefix=prefix, precision=6)
            self.__print_metrics(acc_metrics, prefix=prefix, precision=4, space=2)
                
        return loss_metrics, acc_metrics
            
    def __evaluate_batch(self, imgs, labels):
        imgs = imgs.cuda()
        labels = labels.cuda()
        labels = labels.to(dtype=torch.int64)
        class_labels = labels[:,1]
        joint_labels = labels[:,0] * m.classes + labels[:,1]
        
        # Feedforward
        y_pred = self(imgs)
        
        if self.use_KL:           
            y_class = torch.sum(y_pred, 1)
            y_joint = torch.reshape(y_pred, (-1,2*m.classes))
            
            class_pred_loss = float(torch.nn.NLLLoss()(torch.log(y_class), class_labels))
            joint_pred_loss = float(torch.nn.NLLLoss()(torch.log(y_joint), joint_labels))
            kl_loss = float(KL_Loss(y_pred, m.classes) * self.risk_lambda)
            
            encoder_loss = class_pred_loss + kl_loss
            
            pred_class_count = torch.count_nonzero(torch.argmax(y_class, 1) == class_labels)
            
            loss_metrics = {
                "loss": encoder_loss,
                "kl_loss": kl_loss,
                "classifier_loss": class_pred_loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }
            
        else:          
            pred_class_count = torch.count_nonzero(torch.argmax(y_pred, 1) == class_labels)
            loss = float(torch.nn.NLLLoss()(torch.log(y_pred), class_labels))
            
            loss_metrics = {
                "loss": loss
            }
            
            acc_metrics = {
                "class_acc": float(pred_class_count)
            }

        return loss_metrics, acc_metrics

In [9]:
data_folder = "../Datasets/Experiment"
src_folder = os.path.join(data_folder, "Real World")
target_folder = os.path.join(data_folder, "Product")

d = DataGenerator(
    source_domain=src_folder,
    target_domain=target_folder,
    val_split=0.2,
    test_split=0.2,
    input_shape=(224,224),
    target_labels=0.01,
    target_train=False
)

train_label, train_nlabel = d.train_data()
val = d.val_data()
test = d.test_data()

In [8]:
# Semi-supervised (0.01)
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.01,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=False, risk_lambda=100, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
      
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


KeyboardInterrupt: 

In [27]:
# Semi-supervised (0.01) w/ KL
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.01,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=True, risk_lambda=20, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=True)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


Epoch 1
loss: 0.177308 encoder_loss: 0.10974 kl_loss: 0.064453 classifier_loss: 0.067568 
joint_acc: 0.2537 class_acc: 0.3582 

val_loss: 0.074571 val_kl_loss: 0.03339 val_classifier_loss: 0.041181 
val_class_acc: 0.3182 

Epoch 2
loss: 0.162525 encoder_loss: 0.099491 kl_loss: 0.056398 classifier_loss: 0.063034 
joint_acc: 0.4478 class_acc: 0.4776 

val_loss: 0.0896 val_kl_loss: 0.049528 val_classifier_loss: 0.040072 
val_class_acc: 0.4242 

Epoch 3
loss: 0.161081 encoder_loss: 0.101374 kl_loss: 0.060261 classifier_loss: 0.059707 
joint_acc: 0.4813 class_acc: 0.5037 

val_loss: 0.10817 val_kl_loss: 0.068367 val_classifier_loss: 0.039802 
val_class_acc: 0.4848 

Epoch 4
loss: 0.1535 encoder_loss: 0.097124 kl_loss: 0.058439 classifier_loss: 0.056376 
joint_acc: 0.5933 class_acc: 0.5672 

val_loss: 0.088203 val_kl_loss: 0.048759 val_classifier_loss: 0.039444 
val_class_acc: 0.4545 

Epoch 5
loss: 0.15364 encoder_loss: 0.098677 kl_loss: 0.060103 classifier_loss: 0.054964 
joint_acc: 0.6455

Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


Epoch 1
loss: 0.146327 encoder_loss: 0.080218 kl_loss: 0.034429 classifier_loss: 0.066109 
joint_acc: 0.3134 class_acc: 0.3694 

val_loss: 0.068905 val_kl_loss: 0.028131 val_classifier_loss: 0.040774 
val_class_acc: 0.5 

Epoch 2
loss: 0.140177 encoder_loss: 0.077303 kl_loss: 0.033537 classifier_loss: 0.062873 
joint_acc: 0.4627 class_acc: 0.4813 

val_loss: 0.072675 val_kl_loss: 0.032475 val_classifier_loss: 0.040199 
val_class_acc: 0.5758 

Epoch 3
loss: 0.137385 encoder_loss: 0.079685 kl_loss: 0.039227 classifier_loss: 0.057699 
joint_acc: 0.5448 class_acc: 0.5336 

val_loss: 0.07704 val_kl_loss: 0.041735 val_classifier_loss: 0.035305 
val_class_acc: 0.5606 

Epoch 4
loss: 0.138768 encoder_loss: 0.084123 kl_loss: 0.045446 classifier_loss: 0.054645 
joint_acc: 0.5709 class_acc: 0.5821 

val_loss: 0.107962 val_kl_loss: 0.067441 val_classifier_loss: 0.04052 
val_class_acc: 0.6061 

Epoch 5
loss: 0.141255 encoder_loss: 0.091334 kl_loss: 0.056763 classifier_loss: 0.049921 
joint_acc: 0.6

...

Epoch 1
loss: 0.159871 encoder_loss: 0.089488 kl_loss: 0.043531 classifier_loss: 0.070383 
joint_acc: 0.1306 class_acc: 0.3433 

val_loss: 0.067729 val_kl_loss: 0.02536 val_classifier_loss: 0.042368 
val_class_acc: 0.4394 

Epoch 2
loss: 0.155086 encoder_loss: 0.08891 kl_loss: 0.044492 classifier_loss: 0.066176 
joint_acc: 0.3582 class_acc: 0.4776 

val_loss: 0.089826 val_kl_loss: 0.048382 val_classifier_loss: 0.041444 
val_class_acc: 0.5 

Epoch 3
loss: 0.148558 encoder_loss: 0.086318 kl_loss: 0.044105 classifier_loss: 0.06224 
joint_acc: 0.4925 class_acc: 0.5373 

val_loss: 0.091768 val_kl_loss: 0.055579 val_classifier_loss: 0.036189 
val_class_acc: 0.5303 

Epoch 4
loss: 0.146714 encoder_loss: 0.088015 kl_loss: 0.048794 classifier_loss: 0.058699 
joint_acc: 0.5896 class_acc: 0.6194 

val_loss: 0.092392 val_kl_loss: 0.054549 val_classifier_loss: 0.037843 
val_class_acc: 0.5758 

Epoch 5
loss: 0.150439 encoder_loss: 0.093278 kl_loss: 0.055563 classifier_loss: 0.057161 
joint_acc: 0.60

Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


Epoch 1
loss: 0.158332 encoder_loss: 0.090793 kl_loss: 0.045223 classifier_loss: 0.067539 
joint_acc: 0.2649 class_acc: 0.403 

val_loss: 0.113782 val_kl_loss: 0.074012 val_classifier_loss: 0.039771 
val_class_acc: 0.4697 

Epoch 2
loss: 0.150117 encoder_loss: 0.086776 kl_loss: 0.04326 classifier_loss: 0.063341 
joint_acc: 0.4552 class_acc: 0.5224 

val_loss: 0.076447 val_kl_loss: 0.035906 val_classifier_loss: 0.04054 
val_class_acc: 0.4848 

Epoch 3
loss: 0.14841 encoder_loss: 0.090254 kl_loss: 0.050187 classifier_loss: 0.058156 
joint_acc: 0.5336 class_acc: 0.5597 

val_loss: 0.084339 val_kl_loss: 0.048873 val_classifier_loss: 0.035465 
val_class_acc: 0.4545 

Epoch 4
loss: 0.146339 encoder_loss: 0.091664 kl_loss: 0.054366 classifier_loss: 0.054675 
joint_acc: 0.6269 class_acc: 0.6194 

val_loss: 0.08603 val_kl_loss: 0.048313 val_classifier_loss: 0.037716 
val_class_acc: 0.5606 

Epoch 5
loss: 0.141037 encoder_loss: 0.090649 kl_loss: 0.056539 classifier_loss: 0.050388 
joint_acc: 0.6

Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


Epoch 1
loss: 0.149624 encoder_loss: 0.081595 kl_loss: 0.035979 classifier_loss: 0.068029 
joint_acc: 0.1642 class_acc: 0.2649 

val_loss: 0.067711 val_kl_loss: 0.025757 val_classifier_loss: 0.041954 
val_class_acc: 0.4394 

Epoch 2
loss: 0.156101 encoder_loss: 0.091654 kl_loss: 0.047732 classifier_loss: 0.064446 
joint_acc: 0.3769 class_acc: 0.4403 

val_loss: 0.088684 val_kl_loss: 0.052654 val_classifier_loss: 0.036029 
val_class_acc: 0.5303 

Epoch 3
loss: 0.151757 encoder_loss: 0.091917 kl_loss: 0.051015 classifier_loss: 0.05984 
joint_acc: 0.5373 class_acc: 0.541 

val_loss: 0.084725 val_kl_loss: 0.047977 val_classifier_loss: 0.036747 
val_class_acc: 0.5909 

Epoch 4
loss: 0.145814 encoder_loss: 0.089565 kl_loss: 0.051696 classifier_loss: 0.056249 
joint_acc: 0.5821 class_acc: 0.5896 

val_loss: 0.077818 val_kl_loss: 0.041716 val_classifier_loss: 0.036103 
val_class_acc: 0.5758 

Epoch 5
loss: 0.146107 encoder_loss: 0.094023 kl_loss: 0.059147 classifier_loss: 0.052084 
joint_acc: 

In [25]:
loss, acc = m.evaluate(val)

loss: 0.064281 kl_loss: 0.023097 classifier_loss: 0.041184 
class_acc: 0.5 



In [20]:
# Semi-supervised (0.25)
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.25,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=False, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.023817 
class_acc: 0.4035 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.022466 
class_acc: 0.3509 

Running test  2


...

loss: 0.019324 
class_acc: 0.5088 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.026238 
class_acc: 0.5614 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.022996 
class_acc: 0.4386 

45.263157894736835 7.476237106197374


In [21]:
# Semi-supervised (0.25) w/ KL
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.25,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=True, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.043372 kl_loss: 0.023189 classifier_loss: 0.020182 
class_acc: 0.5614 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.047863 kl_loss: 0.030153 classifier_loss: 0.01771 
class_acc: 0.6316 

Running test  2


...

loss: 0.055563 kl_loss: 0.038938 classifier_loss: 0.016625 
class_acc: 0.6491 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.046911 kl_loss: 0.0273 classifier_loss: 0.019611 
class_acc: 0.4912 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.067079 kl_loss: 0.048827 classifier_loss: 0.018251 
class_acc: 0.6491 

59.64912280701753 6.1778304777750925


In [22]:
# Semi-supervised (0.4)
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.4,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=False, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.016326 
class_acc: 0.6316 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.019363 
class_acc: 0.5439 

Running test  2


...

loss: 0.019644 
class_acc: 0.614 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.022749 
class_acc: 0.4386 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.015867 
class_acc: 0.6316 

57.19298245614036 7.4100744147167985


In [23]:
# Semi-supervised (0.4) w/ KL
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.4,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=True, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.048692 kl_loss: 0.030309 classifier_loss: 0.018383 
class_acc: 0.5789 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.042661 kl_loss: 0.025382 classifier_loss: 0.017279 
class_acc: 0.6316 

Running test  2


...

loss: 0.039603 kl_loss: 0.022554 classifier_loss: 0.017049 
class_acc: 0.6667 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.038365 kl_loss: 0.022904 classifier_loss: 0.015461 
class_acc: 0.6842 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.04148 kl_loss: 0.020299 classifier_loss: 0.021181 
class_acc: 0.4737 

60.70175438596491 7.57439759470979


In [24]:
# Semi-supervised (0.7)
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.7,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=False, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.019813 
class_acc: 0.5789 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.020217 
class_acc: 0.7193 

Running test  2


...

loss: 0.022502 
class_acc: 0.4912 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.016603 
class_acc: 0.6842 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.012666 
class_acc: 0.7193 

63.859649122807014 8.986841034993473


In [29]:
# Semi-supervised (0.7) w/ KL
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=0.7,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=True, risk_lambda=1, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.019904 kl_loss: 0.003167 classifier_loss: 0.016737 
class_acc: 0.6667 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.019611 kl_loss: 0.004195 classifier_loss: 0.015416 
class_acc: 0.7018 

Running test  2


...

loss: 0.019451 kl_loss: 0.003528 classifier_loss: 0.015922 
class_acc: 0.6842 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.022677 kl_loss: 0.004684 classifier_loss: 0.017993 
class_acc: 0.614 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.018521 kl_loss: 0.00377 classifier_loss: 0.014751 
class_acc: 0.7193 

67.71929824561404 3.6125018038550873


In [26]:
# Semi-supervised (1.0)
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=1,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=False, risk_lambda=10, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.017985 
class_acc: 0.6316 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.016293 
class_acc: 0.7018 

Running test  2


...

loss: 0.016969 
class_acc: 0.6491 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.014453 
class_acc: 0.7719 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.017657 
class_acc: 0.614 

67.36842105263159 5.722633835193012


In [28]:
# Semi-supervised (1.0) w/ KL
test_acc = []

for i in range(5):
    print("Running test ", i)
    
    data_folder = "../Datasets/Experiment"
    src_folder = os.path.join(data_folder, "Real World")
    target_folder = os.path.join(data_folder, "Product")

    d = DataGenerator(
        source_domain=src_folder,
        target_domain=target_folder,
        val_split=0.2,
        test_split=0.2,
        input_shape=(224,224),
        target_labels=1.0,
        target_train=False
    )

    train_label, train_nlabel = d.train_data()
    val = d.val_data()
    test = d.test_data()
    
    m = SingleDomainModel(classes=len(d.classes), use_KL=True, risk_lambda=1, lr=1e-4)
    _ = m.cuda()
    m.train(100, train_label, val, patience=5, verbose=False)
    loss, acc = m.evaluate(test)
    test_acc.append(acc["class_acc"])
    
print(np.mean(test_acc) * 100, np.std(test_acc) * 100)

Running test  0


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.01384 kl_loss: 0.004638 classifier_loss: 0.009202 
class_acc: 0.7895 

Running test  1


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.01908 kl_loss: 0.003915 classifier_loss: 0.015165 
class_acc: 0.7018 

Running test  2


...

loss: 0.017607 kl_loss: 0.004642 classifier_loss: 0.012965 
class_acc: 0.7544 

Running test  3


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.017084 kl_loss: 0.003548 classifier_loss: 0.013536 
class_acc: 0.7368 

Running test  4


Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


loss: 0.015197 kl_loss: 0.002888 classifier_loss: 0.012309 
class_acc: 0.807 

75.78947368421052 3.7463432463267767


In [ ]:
epochs = 15

In [209]:
use_KL=True

for p in m.parameters():
    p.requires_grad = True

for i in range(epochs):
    print(f"Epoch {i+1}")
    
    loss, kl_loss, pred_loss, dis_loss, nlab_total = (0,0,0,0,0) 
    pred_count_joint, pred_count_class, dis_count, lab_total = (0,0,0,0)
    
    for batch_label, batch_nlabel in itertools.zip_longest(train_label, train_nlabel):
        
        if batch_label is not None:
            labels = batch_label[1]
            
            if batch_nlabel is not None:
                imgs = torch.cat((batch_label[0], batch_nlabel), 0)
            else:
                imgs = batch_label[0]
        else:
            imgs = batch_nlabel
            labels = None
                
        batch_results = m.run_batch(imgs, labels, use_KL=use_KL, rep_loss=rep_loss, risk_loss=risk_loss)
        loss += batch_results[0]
        kl_loss += batch_results[1]
        pred_loss += batch_results[2]
        dis_loss += batch_results[3]
        pred_count_joint += batch_results[4]
        pred_count_class += batch_results[5]
        dis_count += batch_results[6]
        
        if batch_label is not None:
            lab_total += batch_label[1].shape[0]
        if batch_nlabel is not None:
            nlab_total += batch_nlabel.shape[0]
    total = lab_total + nlab_total
        
    print(f"loss={loss/total}, "
          f"kl_loss={kl_loss/total}, " 
          f"pred_loss={pred_loss/total}, "
          f"dis_loss={dis_loss/total}, "
          f"joint_acc={pred_count_joint/lab_total}, "
          f"class_acc={pred_count_class/lab_total}, "
          f"dis_acc={dis_count/total}"
         )
    print()
    
    loss, kl_loss, pred_loss, dis_loss, total = (0,0,0,0,0) 
    pred_count_joint, pred_count_class, dis_count = (0,0,0)
    
    for data in val:    
        imgs, labels = data   
        batch_results = m.run_batch(imgs, labels, training=False, use_KL=use_KL, rep_loss=rep_loss, risk_loss=risk_loss)
        loss += batch_results[0]
        kl_loss += batch_results[1]
        pred_loss += batch_results[2]
        dis_loss += batch_results[3]
        pred_count_joint += batch_results[4]
        pred_count_class += batch_results[5]
        dis_count += batch_results[6]
        total += data[1].shape[0]  
            
    print(f"val_loss={loss/total}, "
          f"val_kl_loss={kl_loss/total}, " 
          f"val_pred_loss={pred_loss/total}, "
          f"val_dis_loss={dis_loss/total}, "
          f"val_joint_acc={pred_count_joint/total}, "
          f"val_class_acc={pred_count_class/total}, "
          f"val_dis_acc={dis_count/total}"
         )
    print()

    
    loss, kl_loss, pred_loss, dis_loss, total = (0,0,0,0,0) 
    pred_count_joint, pred_count_class, dis_count = (0,0,0)
    for data in test:
        imgs, labels = data
        batch_results = m.run_batch(imgs, labels, training=False, use_KL=use_KL, rep_loss=rep_loss, risk_loss=risk_loss)
        loss += batch_results[0]
        kl_loss += batch_results[1]
        pred_loss += batch_results[2]
        dis_loss += batch_results[3]
        pred_count_joint += batch_results[4]
        pred_count_class += batch_results[5]
        dis_count += batch_results[6]
        total += data[1].shape[0]     
        
    print(f"test_loss={loss/total}, "
          f"test_kl_loss={kl_loss/total}, " 
          f"test_pred_loss={pred_loss/total}, "
          f"test_dis_loss={dis_loss/total}, "
          f"test_joint_acc={pred_count_joint/total}, "
          f"test_class_acc={pred_count_class/total}, "
          f"test_dis_acc={dis_count/total}"
         )
    print()

Epoch 1
loss=0.0448097315912823, kl_loss=0.0031949610505652474, pred_loss=0.029040276423299986, dis_loss=0.012574494669311926, joint_acc=0.21428571428571427, class_acc=0.37714285714285717, dis_acc=0.5984405458089669

val_loss=0.08116570048862033, val_kl_loss=0.0034040855036841498, val_pred_loss=0.030469600359598795, val_dis_loss=0.04729201528761122, val_joint_acc=0.044444444444444446, val_class_acc=0.28888888888888886, val_dis_acc=0.0

test_loss=0.0665176458526076, test_kl_loss=0.004598480044749745, test_pred_loss=0.02362041515216493, test_dis_loss=0.03829875326993173, test_joint_acc=0.03508771929824561, test_class_acc=0.40350877192982454, test_dis_acc=0.0

Epoch 2
loss=0.046705354026883666, kl_loss=0.002489541328673707, pred_loss=0.028486819527534946, dis_loss=0.015728992328309176, joint_acc=0.22285714285714286, class_acc=0.3657142857142857, dis_acc=0.5458089668615984

val_loss=0.0897398206922743, val_kl_loss=0.005053795377413432, val_pred_loss=0.02954330179426405, val_dis_loss=0.0551

loss=0.024398054237718934, kl_loss=0.00020799190613856905, pred_loss=0.005284243922310266, dis_loss=0.018905818375230532, joint_acc=0.8828571428571429, class_acc=0.9314285714285714, dis_acc=0.6257309941520468

val_loss=0.09120824601915148, val_kl_loss=0.0002967918084727393, val_pred_loss=0.037303956349690755, val_dis_loss=0.05360750092400445, val_joint_acc=0.0, val_class_acc=0.6222222222222222, val_dis_acc=0.0

test_loss=0.07948094083551775, test_kl_loss=0.0002799253061152341, test_pred_loss=0.03591607746325041, test_dis_loss=0.043284934863709566, test_joint_acc=0.0, test_class_acc=0.5614035087719298, test_dis_acc=0.0

Epoch 15
loss=0.02136706928295931, kl_loss=8.285282108003227e-05, pred_loss=0.004353846802755639, dis_loss=0.01693036966388918, joint_acc=0.8857142857142857, class_acc=0.9342857142857143, dis_acc=0.6042884990253411

val_loss=0.08339019351535373, val_kl_loss=0.00023287754091951582, val_pred_loss=0.03308609591590034, val_dis_loss=0.050071223576863604, val_joint_acc=0.0, va

In [38]:
class DANN(nn.Module):
    def __init__(
        self, 
        classes=65
    ):
        super(DANN, self).__init__()
        self.classes = classes
        self.encoder = build_encoder()
        self.classifier = build_classifier(512, self.classes, False)
        self.discriminator = build_discriminator(512)
    
    def forward(self, inputs, rep_loss=1):
        features = self.encoder(inputs)
        classes = self.classifier(features)
        domains = self.discriminator(ReverseLayer.apply(features, rep_loss))
        return classes, domains
    
    def run_batch(self, imgs, labels=None, training=True, rep_loss=1):
        if training:
            opt = optim.Adam(self.parameters(), lr=1e-4)
            opt.zero_grad()
            
        class_pred_loss = 0
        pred_count_class = 0
        
        imgs = imgs.cuda()
        y_pred, domain_pred_label = self(imgs) 
        
        if labels is not None:
            labels = labels.cuda()
            labels = labels.to(dtype=torch.int64)
            class_labels = labels[:,1]
            domain_labels = torch.cat((labels[:,0], torch.ones(imgs.shape[0] - labels.shape[0], dtype=torch.long).cuda()), 0)

            y_pred = y_pred[:labels.shape[0]]

            class_pred_loss = torch.nn.NLLLoss()(torch.log(y_pred), class_labels)
            pred_count_class = int(torch.count_nonzero(torch.argmax(y_pred, 1) == class_labels))
             
        dis_loss = torch.nn.NLLLoss()(torch.log(domain_pred_label), domain_labels) * rep_loss
        dis_count = int(torch.count_nonzero(torch.argmax(domain_pred_label, 1) == domain_labels))
        
        loss = class_pred_loss + dis_loss

        if training:
            loss.backward()
            opt.step()
        
        return (
            float(loss),  
            float(class_pred_loss), 
            float(dis_loss), 
            float(pred_count_class),
            float(dis_count)
        )

In [39]:
dann = DANN(classes=len(d.classes))
_ = dann.cuda()

Using cache found in /home/rhc37/.cache/torch/hub/pytorch_vision_v0.6.0


In [40]:
# rep_loss=1

for p in dann.parameters():
    p.requires_grad = True    

for epoch in range(epochs):
    
    print(f"Epoch {epoch+1}")
    
    loss, pred_loss, dis_loss, nlab_total = (0,0,0,0) 
    pred_count_class, dis_count, lab_total = (0,0,0)
    len_dataloader = max(len(train_label), len(train_nlabel))
    
    for i, (batch_label, batch_nlabel) in enumerate(itertools.zip_longest(train_label, train_nlabel)):
        
        p = float(i + epoch * len_dataloader) / epochs / len_dataloader
        rep_loss=2. / (1. + np.exp(-10 * p)) - 1
        
        if batch_label is not None:
            labels = batch_label[1]
            
            if batch_nlabel is not None:
                imgs = torch.cat((batch_label[0], batch_nlabel), 0)
            else:
                imgs = batch_label[0]
        else:
            imgs = batch_nlabel
            labels = None
                
        batch_results = dann.run_batch(imgs, labels, rep_loss=rep_loss)
        loss += batch_results[0]
        pred_loss += batch_results[1]
        dis_loss += batch_results[2]
        pred_count_class += batch_results[3]
        dis_count += batch_results[4]
        
        if batch_label is not None:
            lab_total += batch_label[1].shape[0]
        if batch_nlabel is not None:
            nlab_total += batch_nlabel.shape[0]
    total = lab_total + nlab_total
        
    print(f"loss={loss/total}, "
          f"pred_loss={pred_loss/total}, "
          f"dis_loss={dis_loss/total}, "
          f"class_acc={pred_count_class/lab_total}, "
          f"dis_acc={dis_count/total}"
         )
    print()
    
    loss, pred_loss, dis_loss, total = (0,0,0,0) 
    pred_count_class, dis_count = (0,0)
    
    for data in val:    
        imgs, labels = data   
        batch_results = dann.run_batch(imgs, labels, training=False, rep_loss=rep_loss)
        loss += batch_results[0]
        pred_loss += batch_results[1]
        dis_loss += batch_results[2]
        pred_count_class += batch_results[3]
        dis_count += batch_results[4]
        total += data[1].shape[0]  
            
    print(f"val_loss={loss/total}, "
          f"val_pred_loss={pred_loss/total}, "
          f"val_dis_loss={dis_loss/total}, "
          f"val_class_acc={pred_count_class/total}, "
          f"val_dis_acc={dis_count/total}"
         )
    print()

    
    loss, pred_loss, dis_loss, total = (0,0,0,0) 
    pred_count_class, dis_count = (0,0)
    for data in test:
        imgs, labels = data
        batch_results = dann.run_batch(imgs, labels, training=False, rep_loss=rep_loss)
        loss += batch_results[0]
        pred_loss += batch_results[1]
        dis_loss += batch_results[2]
        pred_count_class += batch_results[3]
        dis_count += batch_results[4]
        total += data[1].shape[0]       
        
    print(f"test_loss={loss/total}, "
          f"test_pred_loss={pred_loss/total}, "
          f"test_dis_loss={dis_loss/total}, "
          f"test_class_acc={pred_count_class/total}, "
          f"test_dis_acc={dis_count/total}"
         )
    print()

Epoch 1
loss=0.029174270220899676, pred_loss=0.028099820627803692, dis_loss=0.00107444961125042, class_acc=0.36826347305389223, dis_acc=0.6510721247563352

val_loss=0.05201121436225043, val_pred_loss=0.02787361145019531, val_dis_loss=0.02413760291205512, val_class_acc=0.4222222222222222, val_dis_acc=0.0

test_loss=0.04142024642542789, test_pred_loss=0.02256692919814796, test_dis_loss=0.018853317227279932, test_class_acc=0.45614035087719296, test_dis_acc=0.0

Epoch 2
loss=0.03042834834513376, pred_loss=0.02361818038464522, dis_loss=0.006810167938703217, class_acc=0.5029940119760479, dis_acc=0.631578947368421

val_loss=0.07134602864583334, val_pred_loss=0.026157898373074, val_dis_loss=0.04518813027275933, val_class_acc=0.4888888888888889, val_dis_acc=0.0

test_loss=0.05892959812231231, test_pred_loss=0.023572536936977452, test_dis_loss=0.03535706118533486, test_class_acc=0.47368421052631576, test_dis_acc=0.0

Epoch 3
loss=0.030507138017092997, pred_loss=0.019705610317096375, dis_loss=0.0

In [81]:
b.shape

torch.Size([64, 2])

In [82]:
labels.shape

torch.Size([32, 2])

In [44]:
loss, kl_loss, pred_loss, dis_loss, total = (0,0,0,0,0) 
pred_count_joint, pred_count_class, dis_count = (0,0,0)
for data in test:
    batch_results = m.run_batch(data, training=False, use_KL=use_KL, rep_loss=rep_loss)
    loss += batch_results[0]
    kl_loss += batch_results[1]
    pred_loss += batch_results[2]
    dis_loss += batch_results[3]
    pred_count_joint += batch_results[4]
    pred_count_class += batch_results[5]
    dis_count += batch_results[6]
    total += data[1].shape[0]

print(f"test_loss={loss/total}, "
      f"test_kl_loss={kl_loss/total}, " 
      f"test_pred_loss={pred_loss/total}, "
      f"test_dis_loss={dis_loss/total}, "
      f"test_joint_acc={pred_count_joint/total}, "
      f"test_class_acc={pred_count_class/total}, "
      f"test_dis_acc={dis_count/total}"
     )

test_loss=0.04912133802447403, test_kl_loss=0.0032100612134264225, test_pred_loss=0.023766850170336272, test_dis_loss=0.022144424287896407, test_joint_acc=0.17543859649122806, test_class_acc=0.24561403508771928, test_dis_acc=0.0


In [45]:
for data in test:
    imgs, labels = data
    break

In [47]:
for batch_label, batch_nlabel in itertools.zip_longest(train_label, train_nlabel):
    imgs_label , labels = batch_label
    imgs_nlabel = batch_nlabel
    break

In [51]:
_, domain_pred = m(imgs_nlabel.cuda())
domain_pred

tensor([0.2744, 0.2950, 0.2825, 0.3014, 0.2855, 0.2858, 0.2895, 0.2864, 0.2892,
        0.2879, 0.2831, 0.2841, 0.2889, 0.2999, 0.2810, 0.2836, 0.2994, 0.2932,
        0.2896, 0.2690, 0.2878, 0.2886, 0.2119, 0.2940, 0.2667, 0.2445, 0.2914,
        0.2854, 0.2884, 0.2849, 0.2927, 0.2858], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [47]:
torch.argmax(torch.sum(y_pred, 1), 1).shape

torch.Size([57])

In [46]:
labels[:,1].shape

torch.Size([57])

In [49]:
torch.sum(labels, axis=1)

tensor([2., 2., 2., 3., 4., 4., 2., 2., 1., 3., 2., 4., 4., 2., 2., 2., 2., 2.,
        1., 3., 2., 1., 3., 4., 1., 3., 4., 1., 4., 3., 2., 1., 3., 4., 4., 4.,
        1., 3., 4., 3., 2., 3., 2., 2., 1., 3., 3., 3., 4., 1., 1., 1., 1., 4.,
        4., 2., 4.])

In [52]:
torch.count_nonzero(torch.argmax(torch.sum(y_pred, 1), 1) == labels[:,1].cuda()) / labels.shape[0]

tensor(0.4386, device='cuda:0')

In [237]:
y_pred = pred[:1]
y_joint = torch.reshape(y_pred, (1, 2*4))
    
y_class = torch.unsqueeze(torch.sum(y_pred, 1), 1)
y_domain = torch.unsqueeze(torch.sum(y_pred, 2), -1)
    
    
y_ind_joint = torch.reshape((y_domain * y_class), (1,2*4))

In [238]:
y_pred

tensor([[[0.2102, 0.3443, 0.0579, 0.0241],
         [0.0946, 0.2194, 0.0309, 0.0186]]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [239]:
y_joint

tensor([[0.2102, 0.3443, 0.0579, 0.0241, 0.0946, 0.2194, 0.0309, 0.0186]],
       device='cuda:0', grad_fn=<ViewBackward>)

In [240]:
y_class

tensor([[[0.3048, 0.5637, 0.0888, 0.0427]]], device='cuda:0',
       grad_fn=<UnsqueezeBackward0>)

In [241]:
y_domain

tensor([[[0.6365],
         [0.3635]]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [242]:
y_ind_joint

tensor([[0.1940, 0.3588, 0.0565, 0.0272, 0.1108, 0.2049, 0.0323, 0.0155]],
       device='cuda:0', grad_fn=<ViewBackward>)

In [219]:
torch.nn.KLDivLoss(log_target=True, reduction="sum")(
    torch.log(y_joint), 
    torch.log(y_ind_joint)
)

tensor(inf)

In [257]:
def kl_divergence(p, q):
    p = p.reshape(-1)
    q = q.reshape(-1)
    print(p, q)
    return sum(p[i] * torch.log2(p[i]/q[i]) for i in range(len(p)))

In [258]:
kl_divergence(y_joint, y_ind_joint)

tensor([0.2102, 0.3443, 0.0579, 0.0241, 0.0946, 0.2194, 0.0309, 0.0186],
       device='cuda:0', grad_fn=<ViewBackward>) tensor([0.1940, 0.3588, 0.0565, 0.0272, 0.1108, 0.2049, 0.0323, 0.0155],
       device='cuda:0', grad_fn=<ViewBackward>)


tensor(0.0046, device='cuda:0', grad_fn=<AddBackward0>)